# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Данные

In [5]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-block alert-info">
    
общая информация    
    
</div>

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-block alert-info">
    
перечень названий столбцов таблицы  
    
</div>

In [7]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**

данные содержат:

1) 21525 записей и 12 признаков

2) целые и вещественные числа, данные типа строка в верхнем и нижнем регистре

3) пропуски типа NaN 

4) категориальные и количественные данные

5) возможны дубликаты

6) проблем в названиях столбцов не выявлено

Каждая строка таблицы содержит данные о заемщиках с детьми и без, разного возраста и трудового стажа, разным уровнем образования. Содержится информация об их семейном положении, указан пол, тип и уровень дохода, а так же цель кредитования.
Данные необходимо привести к единому формату, обработать пропуски, артефакты и другой "Мусор". Особенно важна информация в колонках **children**, **family_status**, **family_status_id**, **debt**, **total_income** и, вероятно, **purpose** для вывода о вероятности погашения кредита в срок с учетом цели кредитования.

## Предобработка данных

### Обработка пропусков

<div class="alert alert-block alert-info">
    
количество пропущенных значений в столбцах
    
</div>

In [4]:
data.isnull().sum() 
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

пропуски типа NaN в одинаковом количестве 2174 имеются в столбцах **days_employed** и **total_income**, что составляет 10,1% от общего количества - удалять не будем, много. Причина пропуска может быть как технологическая, так и в человеческом факторе.

<div class="alert alert-block alert-info">
    
Замена пропусков в столбце **days_employed** на среднее значение
    
1. отрицателные значение приведем к модулю
    
2. исключим стаж более 75 лет (примерно 27000 дней)
    
</div>

In [11]:
data.days_employed = data.days_employed.abs()
days_employed_mean = data[data['days_employed'] < 27000]['days_employed'].mean()
data.days_employed = data.days_employed.fillna(days_employed_mean)

<div class="alert alert-block alert-info">
    
Замена пропусков в столбце **total_income** на среднее значение не подходит - верхние выбросы значительно увеличат показатель, 
    
применим медианное значение
    
</div>

In [12]:
total_income_median = data.total_income.median()
data.total_income = data.total_income.fillna(total_income_median)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

пропуски в данных:

1) определены методом isnull()

2) в столбце **days_employed** заменены на среднее значение методом fillna()

3) в столбце **total_income** заменены на медианы методом fillna()

4) пропусков нет

### Замена типа данных

**2.2.1 столбец children**

<div class="alert alert-block alert-info">
    
поиск артефактов в столбце **children**
    
</div>

In [14]:
data.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

<div class="alert alert-block alert-info">
    
удаление данных со значением более 10, взят модуль чисел меньше 0
    
</div>

In [15]:
data.children = data[data.children < 10]
data.children = data.children.abs()
data.children.value_counts()

0.0    14149
1.0     4865
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

**2.2.2 столбец days_employed**

<div class="alert alert-block alert-info">
    
обработка артефакта - стаж более 75 лет (более 27000 дней)
    
</div>

In [16]:
data[data['days_employed'] >= 27000]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

<div class="alert alert-block alert-info">

большое количество, почти 16%, удалять нельзя, применим нормирование
   
</div>

<div class="alert alert-block alert-info">

определим медиану для заемщиков со стажем:    **0 < days_employed <= 27000** - median_norm_employed
  
</div>

In [17]:
median_norm_employed = data[(data['days_employed'] > 0) & (data['days_employed'] <= 27000)]['days_employed'].median()

<div class="alert alert-block alert-info">

определим медиану для заемщиков со стажем:    **days_employed > 27000** - median_over_days_employed

</div>

In [18]:
median_over_days_employed = data[data['days_employed'] > 27000]['days_employed'].median()

<div class="alert alert-block alert-info">
  
определим понижающий коэф.нормирования:     **days_employed_norm = median_norm_employed / median_over_days_employed**

</div>

In [19]:
days_employed_norm = median_norm_employed / median_over_days_employed
print(days_employed_norm)

0.005367540483916698


<div class="alert alert-block alert-info">

внесем корректировку в стаж **days_employed > 27000:   days_employed = (стаж days_employed > 27000) * days_employed_norm**

</div>

In [20]:
data.loc[data.days_employed > 27000, 'days_employed'] = data.loc[data.days_employed > 27000, 'days_employed'] * days_employed_norm

<div class="alert alert-block alert-info">

меняем столбец **days_employed** на **years_employed** - изменяем стаж в днях на стаж в годах для лучшего восприятия
    
</div>

In [21]:
data.days_employed = data.days_employed / 365
data.set_axis(['children', 'years_employed', 'dob_years', 'education', 'education_id', 
'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
'total_income', 'purpose'], axis = 'columns', inplace = True)

<div class="alert alert-block alert-info">

вещественный тип данных меняем на целочисленный
    
</div>

In [22]:
data.years_employed = data.years_employed.astype('int')

**2.2.3 столбец dob_years**

<div class="alert alert-block alert-info">

определим границы данных по возрасту и устраним артефакты
    
</div>

In [23]:
print(data.dob_years.min())
print(data.dob_years.max())

0
75


<div class="alert alert-block alert-info">

определяем количество заемщиков младше 18 и старше 75 лет
    
</div>

In [24]:
data[data['dob_years'] < 18]['dob_years'].count()

101

In [25]:
data[data['dob_years'] > 75]['dob_years'].count()

0

<div class="alert alert-block alert-info">

доля заемщиков младше 18 лет составляет менее 0.5% - удаляем
    
</div>

In [26]:
data.dob_years = data[data['dob_years'] >= 18]['dob_years']

In [27]:
print(data.dob_years.min())

19.0


**2.2.4 столбец total_income**

<div class="alert alert-block alert-info">

вещественный тип данных меняем на целочисленный по модулю
    
</div>

In [28]:
data.total_income = data.total_income.astype('int').abs() 

**Вывод**

Изменения типов данных: 

- вещественный тип данных изменили на целочисленный

- отрицательные значения взяты по модулю

- устранены артефакты и отклонения - большое количество детей (20), изменен стаж в пересчет на года и исключены заемщики младше 
18 лет

- устранены артефакты стажа: отрицательный - взят модуль, более 75 лет - проведено нормирование


### Обработка дубликатов

Из-за разного форматирования строк в столбцах 'education', 'family_status', 'gender', 'income_type', 'debt', 'purpose' возможны не полные дубликаты. Следует проверить посредством метода value_counts() - покажет уникальные значения и их количество. Применим цикл для списка названий столбцов.

<div class="alert alert-block alert-info">

уникальные значения и количество упоминаний данных в столбцах **education**, **family_status**, **gender**, **income_type**, **debt**, **purpose**
    
</div>

In [29]:
list_duplicates = ['education', 'family_status', 'gender', 'income_type', 'debt', 'purpose']
for row in list_duplicates:
    print(row)
    print('---------------')
    print(data[row].value_counts())
    print('______________________________________________')
    print('______________________________________________')

education
---------------
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
______________________________________________
______________________________________________
family_status
---------------
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
______________________________________________
______________________________________________
gender
---------------
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
_____________

В данных столбцов **education**, **family_status** следует изменить формат на нижний регистр.

Данные столбца **gender** удалить артефакт **XNA**

В данных столбца **purpose** требуется провести лемматизацию.

In [30]:
data.education = data.education.str.lower()
data.family_status = data.family_status.str.lower()

In [31]:
data.gender = data[data.gender != 'XNA']

<div class="alert alert-block alert-info">

количество полных дубликатов
    
</div>

In [32]:
data.duplicated().sum()

103

<div class="alert alert-block alert-info">

удаление полных дубликатов
    
</div>

In [33]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

**Вывод**

причины появления дубликатов:

1) технологическая - данные из разных источников в разном формате и разным типом данных

2) человеческий фактор - данные добавлены без проверки на повторяемость, ошибки при вводе

### Лемматизация

В данных столбца **purpose** требуется провести лемматизацию.

<div class="alert alert-block alert-info">
    
импорт класса Mystem пакета pymystem3

импорт класса Counter() модуля collections
    
получение уникальных значений столбца **purpose**
    
</div>

In [35]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

purpose_list = data['purpose'].unique()

<div class="alert alert-block alert-info">

каждую строчку в **purpose_list** лемматизируем, получаем список лемм и добавляем значения в пустой список **lemms**
    
не понятно - как применить результат **lemmas и Counter(lemmas)** ?
   
</div>

In [36]:
lemmas = []
for i in purpose_list:
    lemma = ' '.join(m.lemmatize(i))
    lemmas.append(lemma)

print(Counter(lemmas)) 

Counter({'автомобиль \n': 2, 'покупка   жилье \n': 1, 'приобретение   автомобиль \n': 1, 'дополнительный   образование \n': 1, 'сыграть   свадьба \n': 1, 'операция   с   жилье \n': 1, 'образование \n': 1, 'на   проведение   свадьба \n': 1, 'покупка   жилье   для   семья \n': 1, 'покупка   недвижимость \n': 1, 'покупка   коммерческий   недвижимость \n': 1, 'покупка   жилой   недвижимость \n': 1, 'строительство   собственный   недвижимость \n': 1, 'недвижимость \n': 1, 'строительство   недвижимость \n': 1, 'на   покупка   подержать   автомобиль \n': 1, 'на   покупка   свой   автомобиль \n': 1, 'операция   с   коммерческий   недвижимость \n': 1, 'строительство   жилой   недвижимость \n': 1, 'жилье \n': 1, 'операция   со   свой   недвижимость \n': 1, 'заниматься   образование \n': 1, 'сделка   с   подержанный   автомобиль \n': 1, 'получение   образование \n': 1, 'свадьба \n': 1, 'получение   дополнительный   образование \n': 1, 'покупка   свой   жилье \n': 1, 'операция   с   недвижимость \

**Вывод**

В наличии значительное количество одинаковых слов, возможно сокращение количества категорий целей

Для получения лемм применен алгоритм:

1. Импорт русскоязычной библиотеки pymystem3

2. Получение списка уникальных строк столбца 'purpose'

3. Лиммитизация каждой строки списка

4. Получаем список лемм

5. Для наглядности итоговый результат соединяем методом join()

6. Добавляем значения в пустой список 'lemms'

7. Проверяем выводом на печать

### Категоризация данных

**Вывод**

<div class="alert alert-block alert-info">

для данных столбца **total_income** требуется категоризация ввиду специфики данных
    
путем разделения доходов на категории **высокий доход**, **средний доход**, **низкий доход**
    
</div>

## Предварительный вывод

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

<div class="alert alert-block alert-info">

количество заемщиков
    
количество случаев задолженности по кредиту
    
процент задолженности по кредиту
    
</div>

In [37]:
number_debitor_children = data.groupby('children')['debt'].count() # количество заемщиков
number_debt_children = data.groupby('children')['debt'].sum() # количество случаев задолженности по кредиту

percent_debt_children = number_debt_children / number_debitor_children * 100 # процент задолженности по кредиту
print(percent_debt_children.sort_values(ascending = False))

children
4.0    9.756098
2.0    9.468033
1.0    9.175258
3.0    8.181818
0.0    7.556693
5.0    0.000000
Name: debt, dtype: float64


**Вывод**

Вероятность задолженности по кредиту самая высокая у заемщиков с 4мя детьми 9.8%
Далее идут с 2мя 9.5% и с 1м 9.2%. Немного ниже у заемщиков с 3мя детьми и составляет 8.2% и заемщиков без детей 7,5%.
У заявителей с 5ю детьми риски равны 0, вероятно из-за низкого количества кредитных историй.
В целом, уровень примерно одинаковый и однозначной зависимости нет. 
Но заявители без детей имеют наименьшую склонность к задолженности.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

<div class="alert alert-block alert-info">

количество заемщиков
    
количество случаев задолженности по кредиту
    
процент задолженности по кредиту
    
</div>

In [38]:
number_debitor_family = data.groupby('family_status')['debt'].count()
number_debt_family = data.groupby('family_status')['debt'].sum()

percent_debt_family = number_debt_family / number_debitor_family * 100
print(percent_debt_family.sort_values(ascending = True))

family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.559272
гражданский брак         9.367455
не женат / не замужем    9.750890
Name: debt, dtype: float64


**Вывод**

Зависимость вероятности задолженности по кредиту от семейного положения следующая по возрастающей риска:

вдовец/вдова 6.6%

в разводе 7.0%

женат / замужем 7.6%

гражданский брак 9.3%

не женат/не замужем 9.7%

Таким образом самая рискованная категория холостые без опыта семейных отношений.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

<div class="alert alert-block alert-info">

для данных столбца **total_income** требуется категоризация ввиду специфики данных
    
путем разделения доходов на категории **высокий доход**, **средний доход**, **низкий доход**
    
</div>

<div class="alert alert-block alert-info">

количество заемщиков
    
количество случаев задолженности по кредиту
    
процент задолженности по кредиту
    
</div>

In [39]:
number_debitor_total_income = data.groupby('total_income')['debt'].count()
number_debt_total_income = data.groupby('total_income')['debt'].sum()

percent_debt_total_income = number_debt_total_income / number_debitor_total_income * 100
print(percent_debt_total_income.sort_values(ascending = False))

total_income
20667      100.0
215276     100.0
101781     100.0
101864     100.0
216240     100.0
           ...  
119472       0.0
119484       0.0
119485       0.0
119488       0.0
2265604      0.0
Name: debt, Length: 18606, dtype: float64


**Вывод**

### Как разные цели кредита влияют на его возврат в срок?

<div class="alert alert-block alert-info">

количество заемщиков
    
количество случаев задолженности по кредиту
    
процент задолженности по кредиту
    
</div>

In [33]:
number_debitor = data.groupby('purpose')['debt'].count()
number_debt = data.groupby('purpose')['debt'].sum()

percent_debt = number_debt / number_debitor * 100
print(percent_debt.sort_values(ascending = False))

purpose
получение дополнительного образования     11.434978
сделка с автомобилем                      11.013216
получение высшего образования             10.798122
сделка с подержанным автомобилем          10.493827
профильное образование                    10.091743
свой автомобиль                           10.041841
заняться образованием                      9.558824
на покупку автомобиля                      9.341826
автомобили                                 9.224319
приобретение автомобиля                    9.110629
на покупку своего автомобиля               9.108911
высшее образование                         8.849558
строительство недвижимости                 8.737864
заняться высшим образованием               8.669355
автомобиль                                 8.519270
получение образования                      8.390023
на проведение свадьбы                      8.376963
дополнительное образование                 8.278867
операции с недвижимостью                   8.148148
свад

**Вывод**

## Общий вывод

<div class="alert alert-block alert-info">

Самыми надежныи заемщиками являются вдовцы/вдовы или разведенные, без детей в основном при покупке недвижимости
    
</div>